In [ ]:
from datasets import load_dataset,Dataset
START_SYM="<--start-->"
# use name="sample-10BT" to use the 10BT sample
# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb", name="sample/10BT", split="train", streaming=True)
# data_files = ["000_00000.parquet","001_00000.parquet","002_00000.parquet",]
# data_files = ["chinese-c4-0000-of-0096.jsonl","chinese-c4-0001-of-0096.jsonl","chinese-c4-0002-of-0096.jsonl","chinese-c4-0003-of-0096.jsonl","chinese-c4-0004-of-0096.jsonl","chinese-c4-0005-of-0096.jsonl",]

# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb/sample/10BT", data_files=data_files,split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')
# fw = load_dataset("/data/bert-chunker-v2/dataset/fineweb/sample/10BT",split="train",num_proc=6,cache_dir='/data/bert-chunker-v2/dataset/cache')
fw = load_dataset("/data/bc-chinese-2/chinese-c4/data",split="train",num_proc=10,cache_dir='/data/bc-chinese-2/dataset/cache')

print(fw)


In [ ]:

print(fw[2]['text'])



## generating data:

In [2]:
def fil_func(x):
    def normalize_newlines(text):
        import re
        
        return re.sub(r'\n{2,}', '\n', text.strip())
    
    
    x_ =  normalize_newlines(x['text'])

    p = len(x_)/x_.count('\n')
    if '\n' in x_ and len(x_)>800 and START_SYM not in x_ and p>200 and len(x_)<2800:
        return True
    return False


d = fw.filter(fil_func,num_proc=10)
print(d)


Filter (num_proc=10): 100%|██████████| 19273007/19273007 [00:52<00:00, 364085.78 examples/s]


Dataset({
    features: ['url', 'timestamp', 'content_language', 'content_type', 'text'],
    num_rows: 758527
})


In [ ]:
print(d[3330]['text'])


In [3]:
def cut(example):
    
    def normalize_newlines(text):
        import re
    
        return re.sub(r'\n{2,}', '\n', text.strip())


    txt =  normalize_newlines( example["text"])
    
    
    
    
    
    START_SYM="<--start-->"
    example["text"] =txt.replace('\n',START_SYM)
    return example

# 使用 map 方法批量处理
d = d.map(cut,num_proc=10)


Map (num_proc=10): 100%|██████████| 758527/758527 [00:24<00:00, 31450.01 examples/s]


In [4]:
d = d.remove_columns(['url', 'timestamp', 'content_language', 'content_type', ])

In [ ]:
print(d[200]['text'])
d

In [6]:
d_train = d.select(range(0, 748527))
d_test = d.select(range(748527, 758527))

In [ ]:
# This code is based on the revised code from fastchat based on tatsu-lab/stanford_alpaca.


from dataclasses import dataclass, field
import json
import time
import logging
import os
from typing import Dict, Optional, List, Union
import torch
from torch.utils.data import Dataset
import re
import torch
from torch import nn
from torch.utils.data import DataLoader
import transformers
from transformers import Trainer, GPTQConfig, AutoModel, AutoTokenizer, AutoConfig
from transformers.trainer_pt_utils import LabelSmoother
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from accelerate.utils import DistributedType
from datasets import load_dataset
import safetensors.torch
import random


IGNORE_TOKEN_ID = -100 
START_SYM = "<--start-->"
max_len = 512
max_use_len = max_len-2

def tokenize_function(item, tokenizer, max_len,):

    chunks=re.split(START_SYM,item['text'])

    # #加上，随机12%去掉句号
    # precess_chunks=[]
    # for c in chunks:
    #     text = c.rstrip()        
    #     chance = random.random()
    #     if chance < 0.12 and text.endswith('.'):
    #         text = text[:-1]+'\n'
    #         precess_chunks.append(text)
    #     else:
    #         precess_chunks.append(c)
    # chunks = precess_chunks


    # chunks=re.split("\n",item['text'])

    input_ids=[]
    labels=[]
    for _,c in enumerate(chunks):
        c=tokenizer(c,truncation=False)['input_ids']
        #去掉头尾，最后两端再加上special tokens在头尾
        c=c[1:-1]
        if c:
            input_ids+=c

            cl=len(c)*[0]

            cl[0]=1
            labels+=cl
    #判断数据是否是空，抛出异常
    if not labels:
        print('--------')
        print(item)
    labels[0]=0

    full_texts = {
            "input_ids": [],
            "labels": [],
            "attention_mask": [],
        }
        
    start_idx = 0

    while(1):

        if start_idx>len(input_ids)-1:
            break

        end_idx = start_idx + max_use_len



        window_input_ids = input_ids[start_idx:end_idx]
        window_labels = labels[start_idx:end_idx]

        # 加入bert 头尾
        window_input_ids = [tokenizer.cls_token_id] + window_input_ids + [tokenizer.sep_token_id]

        window_labels=[IGNORE_TOKEN_ID] + window_labels + [IGNORE_TOKEN_ID]

        # 填充到max_len
        len_now = len(window_input_ids)
        window_input_ids = window_input_ids + [0] * (max_len - len_now)
        window_labels = window_labels + [IGNORE_TOKEN_ID] * (max_len - len_now)
        attention_mask = [1] * len_now + [0] * (max_len - len_now)

        full_texts["input_ids"].append(window_input_ids)
        full_texts["labels"].append(window_labels)
        full_texts["attention_mask"].append(attention_mask)


        label_chunk = labels[start_idx:end_idx]
        last_cut_id = end_idx
        for i in range(len(label_chunk)):
            j= len(label_chunk) - i - 1
            if label_chunk[j]  == 1 and j !=0:
                last_cut_id = start_idx + j
                break

        start_idx = last_cut_id

    return full_texts

tokenizer = AutoTokenizer.from_pretrained(
    "/data/bc-chinese-2/bge-small-zh-v1.5",
    padding_side="right",
    model_max_length=max_len,
    trust_remote_code=True,
)

train_dataset = d_test.map(
    tokenize_function,
    remove_columns=d_test.column_names,
    fn_kwargs={"tokenizer": tokenizer, "max_len": max_len},
    num_proc=10,
)

In [ ]:
train_dataset = train_dataset.to_pandas()
train_dataset = train_dataset.explode(['input_ids',"labels","attention_mask"],ignore_index=True)
print(train_dataset)
train_dataset.to_parquet("/data/bc-chinese-2/newline_10k.parquet")
